In [5]:
from text2brick.dataset import LegoDatasetGenerator
from text2brick.gym import IoUValidityRewardFunc
reward_function = IoUValidityRewardFunc()
generator = LegoDatasetGenerator(reward_function=reward_function, sample_size=1000)

In [6]:
generator.generate_dataset(250)

100%|██████████| 250/250 [01:35<00:00,  2.62it/s]


In [1]:
from text2brick.dataset import LegoPretrainDataset
from torch_geometric.loader import DataLoader 
import torch

BATCH_SIZE = 50

dataset = LegoPretrainDataset("./lego_dataset")
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

c:\Text2Bricks\text2brick\dataset\LegoDataset.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path, map_location='cpu')  # Load file metadata onl

In [5]:
c = 0
for i, batch in enumerate(dataloader):
    #print(batch[4].batch)

    if batch[4].batch.max() < BATCH_SIZE - 1:
        c += 1
        print("Error count: ", c)
        
        print(i)
        print(batch[4].batch)
        print(batch[4].edge_index)
        print(batch[4].x)
print(i)

Error count:  1
262
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1])
tensor([[ 0,  0,  1,  1,  2,  2,  3,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  7,
          7,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15,
         15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 19, 20, 20, 21, 21, 22, 22, 22,
         23, 23, 23, 23, 24, 24, 25, 25, 25, 25, 26, 26, 26, 26, 27, 27, 28, 28,
         28, 29, 29, 29, 30, 30, 30, 31, 31, 32, 32, 33, 33, 34, 35, 35, 36, 36,
         37, 37, 38, 39, 39, 40, 40, 41, 42, 42, 43, 43, 43, 44, 44, 44, 45, 45,
         45, 46, 47, 47, 47, 48, 48, 48, 48, 49, 49, 49, 50, 50, 50, 51, 51, 51,
         52, 52, 52, 53, 53, 53, 54, 54, 54, 55, 55, 55, 56, 56, 56, 57, 57, 57,
         58, 58, 58, 59, 60, 61, 61, 61, 62, 62, 62, 63, 63, 64, 64, 65

In [7]:
print(dataset.__getitem__(85))

Loading samples:   0%|          | 2/3193 [00:36<16:55:44, 19.10s/it]

(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.8044,  ..., -

In [2]:
from text2brick.gym import Text2Brick_v1
model = Text2Brick_v1()

Using cache found in C:\Users\gourd/.cache\torch\hub\pytorch_vision_v0.10.0


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

Using device: cpu


Text2Brick_v1(
  (snn): SNN(
    (cnn): CNN(
      (model): Sequential(
        (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
        (1): ReLU(inplace=True)
        (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
        (3): Fire(
          (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
          (squeeze_activation): ReLU(inplace=True)
          (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
          (expand1x1_activation): ReLU(inplace=True)
          (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (expand3x3_activation): ReLU(inplace=True)
        )
        (4): Fire(
          (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
          (squeeze_activation): ReLU(inplace=True)
          (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
          (expand1x1_activation): ReLU(inplace=True)
          (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), st

In [5]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Define the loss function (CrossEntropyLoss for classification tasks)
x_loss_fn = nn.CrossEntropyLoss(ignore_index=-1)
y_loss_fn = nn.CrossEntropyLoss(ignore_index=-1)

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def train_model(model, dataloader, optimizer, x_loss_fn, y_loss_fn, num_epochs=10):
    model.train()  # Set the model to training mode

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        
        # Loop over the batches of data
        for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            print("loaded")
            optimizer.zero_grad()  # Zero the parameter gradients

            image_target = batch[0].to(device)
            current_build_img = batch[1].to(device)
            next_node = batch[2].to(device)
            reward = batch[3].to(device)
            current_graph = batch[4].to(device)
            
            x_pred, y_pred = model(current_build_img, current_graph, reward, image_target=image_target, return_logits=True)
            x_true, y_true = next_node[:, 0], next_node[:, 1]

            # Compute loss for both x and y coordinates
            x_loss = x_loss_fn(x_pred, x_true)
            y_loss = y_loss_fn(y_pred, y_true)

            # Combine the losses (you can weight them differently if needed)
            loss = x_loss + y_loss
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        # Print epoch loss
        #print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Run the training loop
train_model(model, dataloader, optimizer, x_loss_fn, y_loss_fn, num_epochs=10)


Loading samples: 4145it [08:10,  2.27s/it]c:\Text2Bricks\text2brick\dataset\LegoDataset.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path, map

loaded


Loading samples: 4241it [08:17, 26.61it/s]

loaded


Loading samples: 4293it [08:23, 22.19it/s]

loaded


Epoch 1/10:   3%|▎         | 2/64 [00:16<08:30,  8.23s/it]


KeyboardInterrupt: 

Loading samples: 4294it [08:40, 22.19it/s]